In [2]:
import re
import os
import pandas as pd

In [3]:
def custo_por_mao(big_blind,small_blind,ante,numero_de_jogadores):
    return (big_blind + small_blind + ante*numero_de_jogadores)/numero_de_jogadores

def convert_k_to_numeric(df, column_name):
    """
    Converts values in a specified column from 'K' notation to numeric.

    Parameters:
    df (pd.DataFrame): The DataFrame containing the column to be converted.
    column_name (str): The name of the column to convert.

    Returns:
    pd.DataFrame: The DataFrame with the specified column converted.
    """
    def convert(value):
        if isinstance(value, str) and 'K' in value:
            return int(float(value.replace('K', '')) * 1000)
        return value

    df[column_name] = pd.to_numeric(df[column_name].apply(convert))
    return df

In [4]:
blind_structure = pd.read_csv('./1_dolar_45_players_sit_go.csv')
blind_structure[['small_blind','big_blind']] = blind_structure['Blinds'].str.split('/',expand=True)
blind_structure.set_index('Level',inplace=True)

blind_structure = convert_k_to_numeric(blind_structure,'Ante')
blind_structure = convert_k_to_numeric(blind_structure,'small_blind')
blind_structure = convert_k_to_numeric(blind_structure,'big_blind')
blind_structure['Minutes'] = pd.to_numeric(blind_structure['Minutes'])

#### A cada volta completa na mesa de tamanho 9 numero de jogadores no torneio o custo médio por mão é dado por:

#### (N * Ante + bigblind + smalllind)/N ou Ante + (bigblind + smalllind)/N

In [75]:
estatistica_de_tempo_por_mao = pd.read_csv('./tempo_por_mao_n_jogadores.csv',index_col=0)

In [76]:
estatistica_de_tempo_por_mao

,count,mean,std,min,25%,50%,75%,max
numero_de_players,,,,,,,,
2,352.0,14.803977,10.388943,1.0,7.00,12.0,20.0,68.0
3,296.0,21.043919,14.585624,4.0,10.00,18.0,27.0,100.0
4,125.0,23.984000,13.906360,7.0,13.00,21.0,30.0,68.0
5,234.0,30.529915,16.929834,7.0,18.00,26.0,39.0,98.0
6,534.0,38.189139,19.685896,8.0,23.25,34.0,49.0,105.0
7,620.0,44.612903,19.835148,8.0,29.75,41.0,56.0,106.0
8,1451.0,50.235010,21.517224,11.0,33.00,47.0,64.5,108.0
9,557.0,55.454219,22.154708,13.0,38.00,53.0,71.0,109.0


In [5]:
blind_structure['custo_medio_por_mao'] = blind_structure['Ante'] + (blind_structure['small_blind'] + blind_structure['big_blind'])/9
blind_structure['Tempo Acumulado'] = blind_structure['Minutes'].cumsum()

In [6]:
df_transformed = pd.DataFrame({
    'Level': blind_structure.index.repeat(10).values,
    'Value': range(1, blind_structure['Tempo Acumulado'].max() + 1)
}).set_index("Level")


In [87]:
blind_structure_extendido = pd.merge(df_transformed,blind_structure,left_index=True,right_index=True,how='left')
blind_structure_extendido = blind_structure_extendido.drop("Tempo Acumulado",axis=1)
blind_structure_extendido['maos_jogadas_por_minuto'] = 60/45
blind_structure_extendido['custo_por_minuto'] = blind_structure_extendido['maos_jogadas_por_minuto'] * blind_structure_extendido['custo_medio_por_mao']
blind_structure_extendido['custo_acumulado'] = blind_structure_extendido.custo_por_minuto.cumsum()

In [112]:
custo_final = blind_structure_extendido[blind_structure_extendido['Value'] == 95].iloc[0]['custo_acumulado']
custo_inicial = blind_structure_extendido[blind_structure_extendido['Value'] >= 1].iloc[0]['custo_acumulado']

In [113]:
quantidade_necessaria_de_fichas = custo_final - custo_inicial

In [114]:
quantidade_necessaria_de_fichas/(1500*45)

0.08573497942386835